In [1]:
# !pip3 install holidays

In [12]:
import pandas as pd
import holidays
import datetime
import calendar

In [3]:
df = pd.read_csv('/home/faye/data/vehicles_2018_cleaned_dataset')

In [4]:
# Converting date columns to datetime
datetime_columns = ['DAYOFSERVICE', 'LASTUPDATE']
for column in datetime_columns:
    df[column] = pd.to_datetime(df[column])

In [5]:
# Getting the Irish holidays - including bank holidays
# https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
irish_holidays_2018 = []
for date in holidays.Ireland(years=2018).items():
    irish_holidays_2018.append(str(date[0]))

In [6]:
irish_holidays_2018

['2018-01-01',
 '2018-12-25',
 '2018-03-17',
 '2018-03-19',
 '2018-04-02',
 '2018-05-07',
 '2018-06-04',
 '2018-08-06',
 '2018-10-29',
 '2018-12-26']

In [14]:
df['Is_HOLIDAY'] = [1 if str(val).split()[0] in irish_holidays_2018 else 0 for val in df['DAYOFSERVICE']]
df['IS_WEEKDAY'] = [1 if int(val.weekday()) < 5 else 0 for val in df['DAYOFSERVICE']]
df['DAYOFWEEK'] = [calendar.day_name[val.weekday()] for val in df['DAYOFSERVICE']]
df['MONTHOFSERVICE'] = df['DAYOFSERVICE'].dt.strftime('%B')

In [17]:
df.tail(5)

,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,Is_Holiday,Is_Weekday,DayOfWeek
272617,2018-12-29,3393878,264237,0 days 17:18:40,2019-01-16 18:00:42,0,0,Saturday
272618,2018-12-29,3394105,250335,0 days 14:30:54,2019-01-16 18:00:42,0,0,Saturday
272619,2018-12-29,3394109,172539,0 days 12:19:09,2019-01-16 18:00:42,0,0,Saturday
272620,2018-12-29,3394130,188057,0 days 10:49:08,2019-01-16 18:00:42,0,0,Saturday
272621,2018-12-29,3394131,291697,0 days 17:41:17,2019-01-16 18:00:42,0,0,Saturday
